In [1]:
#Par Loïc Lotito

#chargement des bibliothéque
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [2]:
#Charger le fichier xls customers
xls_file = pd.ExcelFile('customers.xls')
xls_file

In [3]:
# Affiche les feuilles
xls_file.sheet_names

['customers_apres_rfm', 'Graph']

In [1]:
# Séléction de la feuille à traiter et définition du dataframe
#Affichage des 5 premiéres lignes


In [5]:
customersdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6516 entries, 0 to 6515
Data columns (total 33 columns):
NumCli                     6516 non-null object
Compte_transaction         6516 non-null int64
Total CA                   6516 non-null int64
Score_recence              6516 non-null int64
Score_frequence            6516 non-null int64
Score_monetaire            6516 non-null int64
Score_RFM                  6516 non-null int64
BU                         6508 non-null object
Nom du compte              6507 non-null object
SIRET                      5712 non-null object
Code NAF                   1884 non-null object
Effectif de la sociÃ©te    5092 non-null object
Code postal                6486 non-null object
Dep                        6486 non-null object
Pays Navision              6502 non-null object
Date de newbizz            1571 non-null object
SOLIDWORKS ELECTRICAL      6516 non-null int64
VISIATIV                   6516 non-null int64
VDOC                       6516 non-n

In [6]:
# Nous supprimons colonnes inutiles , ces colonnes ne seront pas utiles dans notre clustering
customersdf = customersdf.drop(['BU','Nom du compte','SIRET','Code NAF','Effectif de la sociÃ©te','Code postal','Dep','Pays Navision','Date de newbizz','SOLIDWORKS ELECTRICAL','VISIATIV','VDOC','SOLIDWORKS EDUC','SIMULIA','NOVAXEL','ISOTOOLS','ENOVIA','DELMIA','CUSTOMTOOLS','COMMERCE','CATIA','AUTRES CAO','3DVIA','SOLIDWORKS TECH','SOLIDWORKS','DateFacture'], axis=1)

In [7]:
# Analyse statistique sur les attributs séléctionnés  
customersdf.describe()

,Compte_transaction,Total CA,Score_recence,Score_frequence,Score_monetaire,Score_RFM
count,6516.000000,6516.000000,6516.000000,6516.000000,6516.000000,6516.000000
mean,3.353131,14912.877839,3.000460,2.917894,3.005832,332.230816
std,4.371793,26826.389521,1.414485,1.403596,1.412681,142.015741
min,1.000000,-3704.000000,1.000000,1.000000,1.000000,121.000000
25%,1.000000,5178.500000,2.000000,2.000000,2.000000,223.000000
50%,2.000000,7326.000000,3.000000,3.000000,3.000000,333.000000
75%,4.000000,15424.500000,4.000000,4.000000,4.000000,444.000000
max,120.000000,785469.000000,5.000000,5.000000,5.000000,555.000000


In [8]:
# Sélection des attributs avant le clustering
customersdf = customersdf[[u'Score_monetaire', u'Score_recence', u'Score_frequence']]
print (customersdf)

      Score_monetaire  Score_recence  Score_frequence
0                   5              4                1
1                   5              1                2
2                   4              1                4
3                   4              3                4
4                   3              1                2
5                   5              2                3
6                   4              2                5
7                   4              4                3
8                   3              1                4
9                   1              4                3
10                  1              5                1
11                  3              4                5
12                  4              5                1
13                  1              5                2
14                  1              1                2
15                  3              1                5
16                  5              5                5
17                  5       

In [9]:
# importation de l'algorithme
from sklearn.cluster import KMeans
clustering_model = KMeans(n_clusters=8,precompute_distances=True)

In [10]:
# affichage du temps de traitement
%time clusters = clustering_model.fit_predict(customersdf)

Wall time: 216 ms


In [11]:
#Afficher les cluster
centroids = clustering_model.cluster_centers_
print(centroids)

[[ 3.99767442  2.06046512  4.53255814]
 [ 2.06168549  1.62901825  2.23284101]
 [ 4.26869806  3.88088643  1.10526316]
 [ 3.999       4.481       3.859     ]
 [ 1.4684432   4.38008415  3.80084151]
 [ 1.5061296   2.05779335  4.5323993 ]
 [ 4.50977836  1.71577575  2.27640156]
 [ 1.81420765  4.1079235   1.15300546]]


In [12]:
#affichage de l'inertie
print (clustering_model.inertia_)

8733.55203159


In [13]:
#affichage de la silhouette 
from sklearn.metrics import silhouette_score 
silhouette = silhouette_score(customersdf.values, clusters, metric='euclidean', sample_size=2000) 
print ("Silhouette score :", silhouette)

Silhouette score : 0.35223383446


In [14]:
#Création et affichage du dataframe après clustering
final = customersdf.join(pd.Series(clusters, index=customersdf.index, name='cluster'))
final['cluster'] = final['cluster'].map(lambda cluster_id: 'cluster' + str(cluster_id))
print (final.head)

<bound method NDFrame.head of       Score_monetaire  Score_recence  Score_frequence   cluster
0                   5              4                1  cluster2
1                   5              1                2  cluster6
2                   4              1                4  cluster0
3                   4              3                4  cluster0
4                   3              1                2  cluster1
5                   5              2                3  cluster6
6                   4              2                5  cluster0
7                   4              4                3  cluster3
8                   3              1                4  cluster0
9                   1              4                3  cluster4
10                  1              5                1  cluster7
11                  3              4                5  cluster3
12                  4              5                1  cluster2
13                  1              5                2  cluster7
14        

In [15]:
#Affichage de la taille des clusters
taille = pd.DataFrame({'taille': final['cluster'].value_counts()})
print (taille)

          taille
cluster1    1151
cluster3    1000
cluster0     860
cluster6     767
cluster7     732
cluster2     722
cluster4     713
cluster5     571


In [16]:
#Préparation du fichier final avant téléchargement
customersid = xls_file.parse('customers_apres_rfm')
customersid = customersid.drop(['Score_monetaire', u'Score_recence', u'Score_frequence'], axis=1)
final_clustering = pd.concat([customersid, final], axis=1)
final_clustering = final_clustering[[u'NumCli',u'Score_monetaire', u'Score_recence', u'Score_frequence','cluster']]
print (final_clustering)

          NumCli  Score_monetaire  Score_recence  Score_frequence   cluster
0       CL000004                5              4                1  cluster2
1       CL000017                5              1                2  cluster6
2       CL000035                4              1                4  cluster0
3       CL000050                4              3                4  cluster0
4       CL000062                3              1                2  cluster1
5       CL000071                5              2                3  cluster6
6       CL000083                4              2                5  cluster0
7       CL000097                4              4                3  cluster3
8       CL000107                3              1                4  cluster0
9       CL000119                1              4                3  cluster4
10      CL000130                1              5                1  cluster7
11      CL000151                3              4                5  cluster3
12      CL00

In [17]:
#DATAVIZ 1/2 : PREPARATION
avant_viz = final_clustering.groupby(['cluster',]).aggregate(np.mean).reset_index()
print (avant_viz)

    cluster  Score_monetaire  Score_recence  Score_frequence
0  cluster0         3.997674       2.060465         4.532558
1  cluster1         2.061685       1.629018         2.232841
2  cluster2         4.268698       3.880886         1.105263
3  cluster3         3.999000       4.481000         3.859000
4  cluster4         1.468443       4.380084         3.800842
5  cluster5         1.506130       2.057793         4.532399
6  cluster6         4.509778       1.715776         2.276402
7  cluster7         1.814208       4.107923         1.153005


In [ ]:
#DATAVIZ 2/2 : GRAPHIQUE

# Réglage de la positions et la largeur pour les barres
pos = list(range(len(avant_viz['Score_monetaire'])))
width = 0.25

# Traçage des barres
fig, ax = plt.subplots(figsize=(14,5))

# Créer ue bar avec les données de Score monétaire ,
# En position pos ,
plt.bar(pos,
        #Utilisant Avant_viz['score_monétaire'] comme données,
        avant_viz['Score_monetaire'],
        # Réglage de la largeur
        width,
        # Réglage de l'alpha
        alpha=0.5,
        # Réglage de la couleur
        color='#EE3224',
        # Réglage de l'étiquette
        label=avant_viz['cluster'][0])

# Créer ue bar avec les données de Score récence ,
plt.bar([p + width for p in pos],
        avant_viz['Score_recence'],
        width,
        alpha=0.5,
        color='#F78F1E',
        label=avant_viz['cluster'][1])

# Créer ue bar avec les données de Score fréquence ,
plt.bar([p + width*2 for p in pos],
        avant_viz['Score_frequence'],
        width,
        alpha=0.5,
        color='#FFC222',
        label=avant_viz['cluster'][2])

# Définition de l'étiquette de l'axe y
ax.set_ylabel('Score')

# Titre du graphique
ax.set_title('Médiane score RFM par Cluster')

# Réglage de la position des étiquettes sur l'axe x
ax.set_xticks([p + 1.5 * width for p in pos])

# Définition des étiquettes pour l'axe x
ax.set_xticklabels(avant_viz['cluster'])

# Réglage des limites de l'axe x et axe y
plt.xlim(min(pos)-width, max(pos)+width*4)
plt.ylim([0, max(avant_viz['Score_monetaire'] + avant_viz['Score_recence'] + avant_viz['Score_recence'])] )

# Ajout d'une légende
plt.legend(['Score_monetaire', 'Score_recence', 'Score_frequence'], loc='upper left')
plt.grid()
plt.show()

In [ ]:
#Export des données
avant_viz.to_csv('clients_RFM_mediane_cluster')
final_clustering.to_csv('clients_RFM_clustering_Kmeans')

In [ ]:
rom sklearn2pmml import sklearn2pmml

sklearn2pmml(clustering_model, "LogisticRegressionIris.pmml", with_repr = True)